In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

import re
from datetime import datetime 
from dateutil import parser

import pandas as pd
import numpy as np
from collections import defaultdict


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.support import expected_conditions as EC
import time

import re
from datetime import datetime 
from dateutil import parser

import pandas as pd
import numpy as np
from collections import defaultdict



In [2]:
def clean_string(s):
    return s.lower().replace(" ", " ").replace("-", " ").replace(".", "").replace("'","")

def split_comma(text):
    text_red, text_blue = text.split(",", 1)
    before_comma = text_red.strip()
    after_comma = text_blue.strip()
    return before_comma, after_comma

def scrape_upcoming(driver, event_rows):
    upcoming_dic = defaultdict(list)    
    event_links = [event_row.find_element(By.CSS_SELECTOR, "a").get_attribute("href") for event_row in event_rows]

    for event_link in event_links:
        driver.get(event_link)
        table_body = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.CLASS_NAME, "b-fight-details__table-body"))
            )
        
        current_event_date = driver.find_element(By.CSS_SELECTOR, ".b-list__box-list-item").text.replace('DATE:','')
        event_date = parser.parse(current_event_date).date()
        event_name = driver.find_element(By.CSS_SELECTOR, ".b-content__title-highlight").text
        location = driver.find_element(By.CSS_SELECTOR, ".b-list__box-list-item:nth-child(2)").text.replace('LOCATION:','')
        fights = table_body.find_elements(By.CLASS_NAME, "b-fight-details__table-row")

        for fight in fights: #navigating the specific fights 

            columns = fight.find_elements(By.CLASS_NAME, "b-fight-details__table-col")
            fighter_names = columns[1].text.replace("\n", ", ")
            weight_class = columns[6].text
            red_fighter, blue_fighter = split_comma(fighter_names)
            fighter_links = columns[1].find_elements(By.TAG_NAME, 'a')

            heights = []
            reaches = []
            dobs = []

            for link in fighter_links:
                url = link.get_attribute("href")
                driver.get(url)
                height_li = driver.find_element(By.XPATH, "//li[i[normalize-space(text())='Height:']]")
                reach_li = driver.find_element(By.XPATH, "//li[i[normalize-space(text())='Reach:']]")
                dob_li = driver.find_element(By.XPATH, "//li[i[normalize-space(text())='DOB:']]")
                
                height = height_li.text.replace("HEIGHT:", "").strip()  
                reach = reach_li.text.replace("REACH:", "").strip()    
                dob = dob_li.text.replace("DOB:", "").strip()  

                heights.append(height)
                reaches.append(reach)
                dobs.append(dob)
                driver.back()

            upcoming_dic['red_fighter'].append(red_fighter)
            upcoming_dic['blue_fighter'].append(blue_fighter)
            upcoming_dic['weight_class'].append(weight_class)
            upcoming_dic['event_date'].append(event_date)
            upcoming_dic['reach_red'].append(reaches[0])
            upcoming_dic['reach_blue'].append(reaches[1])
            upcoming_dic['height_red'].append(heights[0])
            upcoming_dic['height_blue'].append(heights[1])
            upcoming_dic['dob_red'].append(dobs[0])
            upcoming_dic['dob_blue'].append(dobs[1])
    
    upcoming_df = pd.DataFrame(upcoming_dic, columns=['red_fighter','blue_fighter', 'weight_class','event_date', 'reach_red','reach_blue',
                                                      'height_red','height_blue','dob_red','dob_blue'])
    return upcoming_df


In [ ]:

def scrape_ufc(end_date, get_upcoming):
    """end date: year-month-date ('2025-01-01'), set to None if all data needs scraping"""
    
    if end_date is not None:
        end_date = datetime.strptime(end_date, "%Y-%m-%d").date()
    else:
        end_date = None

    global_df = pd.DataFrame()
    
    driver = webdriver.Chrome()

    # driver.get("http://ufcstats.com/statistics/events/completed?page=all")

    base_url = 'http://ufcstats.com/statistics/events/completed?page='

    start_page = 1
    end_page = 28  # Adjust this number to the last page number (or dynamically calculate)

    # Loop through each page
    for page_num in range(start_page, end_page + 1):
        page_url = f"{base_url}{page_num}"  # Build the URL for the current page
        print(f"Navigating to page {page_num}: {page_url}")
        
        # Navigate to the page
        driver.get(page_url)
    
        # get upcoming card with no fight results, return that data 
        if get_upcoming == True:
            event_rows = WebDriverWait(driver, 1).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".b-statistics__table-row_type_first"))
            )
            upcoming_df = scrape_upcoming(driver, event_rows)
            return upcoming_df
        
        # else, continue getting all data 
        else:
            event_rows = WebDriverWait(driver, 1).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".b-statistics__table-row"))
            )
            event_rows = event_rows[2:]

        # outer page, iterate through events table 
        event_links = [event_row.find_element(By.CSS_SELECTOR, "a").get_attribute("href") for event_row in event_rows]

        # navigate to a specific link 
        for event_link in event_links:
            driver.get(event_link)
            table_body = WebDriverWait(driver, 1).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "b-fight-details__table-body"))
                )
            
            current_event_date = driver.find_element(By.CSS_SELECTOR, ".b-list__box-list-item").text.replace('DATE:','')
            event_date = parser.parse(current_event_date).date()
            # event_date = datetime.strptime(date_str, "%Y-%m-%d").date()

            event_name = driver.find_element(By.CSS_SELECTOR, ".b-content__title-highlight").text
            location = driver.find_element(By.CSS_SELECTOR, ".b-list__box-list-item:nth-child(2)").text.replace('LOCATION:','')

            # iterate through individual fights 
            fights = table_body.find_elements(By.CLASS_NAME, "b-fight-details__table-row")
            for fight in fights: #navigating the specific fights 
                
                columns = fight.find_elements(By.CLASS_NAME, "b-fight-details__table-col")

                # Assume winner always row 0 
                # winner_loser = columns[1].text.replace("\n", ", ") #FIX ISSUE WITH NO CONTESTS 
                # winner, _ = split_comma(winner_loser)

                weight_class = columns[6].text
                method = columns[7].text.replace("\n", ", ")
                round = columns[8].text

                #going to the fight specific page 
                fight_link = fight.get_attribute("onclick")

                if fight_link:
                    fight_url = fight_link.split("'")[1] 

                    # Navigate to the fight details page
                    driver.get(fight_url)

                    table_body_totals = WebDriverWait(driver, 1).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "b-fight-details__table-body"))
                    )            
                    rows = table_body_totals.find_elements(By.CLASS_NAME, "b-fight-details__table-row")
                    columns = rows[0].find_elements(By.CLASS_NAME, "b-fight-details__table-col")

                    # TOTALS TABLE, replace newline(\n) with comma, split by comma, red is 0, blue is 1 
                    # fighter_names = columns[0].text.replace("\n", ", ") 
                    kd = columns[1].text.replace("\n", ", ")             
                    sig_str = columns[2].text.replace("\n", ", ")        
                    sig_str_percent = columns[3].text.replace("\n", ", ")  
                    total_str = columns[4].text.replace("\n", ", ")     
                    td = columns[5].text.replace("\n", ", ")            
                    td_percent = columns[6].text.replace("\n", ", ")     
                    sub_att = columns[7].text.replace("\n", ", ")        
                    rev = columns[8].text.replace("\n", ", ")            
                    ctrl = columns[9].text.replace("\n", ", ") 

                    kd_red, kd_blue = split_comma(kd)
                    sig_str_red, sig_str_blue = split_comma(sig_str)
                    str_red, str_blue = split_comma(sig_str_percent)
                    td_red, td_blue = split_comma(td)
                    td_pct_red, td_pct_blue = split_comma(td_percent)
                    strikes_red, strikes_blue = split_comma(total_str)
                    sub_att_red, sub_att_blue = split_comma(sub_att)
                    rev_red, rev_blue = split_comma(rev)
                    ctrl_red, ctrl_blue = split_comma(ctrl)

                    win_bonus = 0
                    fight_bonus = 0

                    perf_bonus_exists = bool(driver.find_elements("xpath", "//img[contains(@src, 'perf.png')]"))
                    fight_bonus_exists = bool(driver.find_elements("xpath", "//img[contains(@src, 'fight.png')]"))

                    if perf_bonus_exists: 
                        win_bonus = 1
                    if fight_bonus_exists: 
                        fight_bonus = 1

                    # GET LINK TO FIGHTERS PAGE 
                    fighters = columns[0].find_elements(By.TAG_NAME, 'a')
                    heights = []
                    reaches = []
                    stances = []
                    records = []
                    dobs = []
                    
                    # NAVIGATE TO FIGHTERS PAGE 
                    for link in fighters:
                        url = link.get_attribute("href") 
                        driver.get(url)

                        # check for bonus 
                        # try:
                        #     per_bonus = driver.find_element("xpath", "//img[contains(@src, 'perf.png')]")
                        #     print("perf.png exists on the page!")
                        # except NoSuchElementException:
                        #     print("perf.png not found on the page.")

                        # try:
                        #     driver.find_element("xpath", "//img[contains(@src, 'fight.png')]")
                        #     print("fight.png exists on the page!")
                        # except NoSuchElementException:
                        #     print("fight.png not found on the page.")
     
                        height_li = driver.find_element(By.XPATH, "//li[i[normalize-space(text())='Height:']]")
                        reach_li = driver.find_element(By.XPATH, "//li[i[normalize-space(text())='Reach:']]")
                        stance_li = driver.find_element(By.XPATH, "//li[i[normalize-space(text())='STANCE:']]")
                        dob_li = driver.find_element(By.XPATH, "//li[i[normalize-space(text())='DOB:']]")
                        record_element = driver.find_element(By.XPATH, "//span[@class='b-content__title-record']")

                        height = height_li.text.replace("HEIGHT:", "").strip()  
                        reach = reach_li.text.replace("REACH:", "").strip()    
                        stance = stance_li.text.replace("STANCE:", "").strip() 
                        dob = dob_li.text.replace("DOB:", "").strip()  
                        record = record_element.text.replace("RECORD:", "").strip()  

                        heights.append(height)
                        reaches.append(reach)
                        stances.append(stance)
                        records.append(record)
                        dobs.append(dob)
                        driver.back()

                    # SIG STRIKES TABLE
                    tables = driver.find_elements(By.CLASS_NAME, "b-fight-details__table-body")

                    sig_strikes = tables[2]
                    rows = sig_strikes.find_elements(By.CLASS_NAME, "b-fight-details__table-row")
                    columns = rows[0].find_elements(By.TAG_NAME, "td")

                    head = columns[3].text.replace("\n", ", ")
                    body = columns[4].text.replace("\n", ", ")
                    leg = columns[5].text.replace("\n", ", ")
                    distance = columns[6].text.replace("\n", ", ")
                    clinch = columns[7].text.replace("\n", ", ")
                    ground = columns[8].text.replace("\n", ", ")

                    head_red, head_blue = split_comma(head)
                    body_red, body_blue = split_comma(body)
                    leg_red, leg_blue = split_comma(leg)
                    distance_red, distance_blue = split_comma(distance)
                    clinch_red, clinch_blue = split_comma(clinch)
                    ground_red, ground_blue = split_comma(ground)

                    red_fighter = driver.find_element(By.XPATH, "//i[@class='b-fight-details__charts-name b-fight-details__charts-name_pos_left js-chart-name' and @data-color='red']").text.strip()
                    blue_fighter = driver.find_element(By.XPATH, "//i[@class='b-fight-details__charts-name b-fight-details__charts-name_pos_right js-chart-name' and @data-color='blue']").text.strip()
                    
                    # names and statues under same class, so name will correspond to correct status 
                    names = driver.find_elements("xpath", "//a[contains(@class, 'b-fight-details__person-link')]")
                    statuses = driver.find_elements("xpath", "//i[contains(@class, 'b-fight-details__person-status')]")

                    fighter_element = driver.find_element("xpath", "//i[contains(@class, 'b-fight-details__person-status')]")
                    winner = ''
                    if "nc" in fighter_element.text.lower():
                        winner = 'NC'

                    for name_el, status_el in zip(names, statuses):
                        name = name_el.text.strip()
                        status = status_el.text.strip()

                        if str(status).lower() == 'w':

                            if str(name).lower() == str(red_fighter).lower():
                                winner = red_fighter
                            
                            else: 
                                winner = blue_fighter
                    
                    if winner == '':
                        winner = 'DRAW'

                    row = {
                        'event_name':event_name, 'event_date':event_date, 'event_location':location, 'fight_url':fight_url,
                        
                        'red_fighter':red_fighter, 'blue_fighter':blue_fighter, 'red_record':records[0], 'blue_record':records[1],'dob_red':dobs[0], 'dob_blue':dobs[1],
                        'height_red':heights[0], 'height_blue':heights[1], 'reach_red':reaches[0], 'reach_blue':reaches[1], 'stance_red':stances[0], 'stance_blue':stances[1],

                        'performance_bonus_winner':win_bonus, 'fight_otn_bonus':fight_bonus,
                        
                        'weight_class':weight_class, 'method':method, 'round':round,

                        'kd_red':kd_red, 'kd_blue':kd_blue, 
                        'sig_str_red':sig_str_red, 'sig_str_blue':sig_str_blue, 'sig_str_percent_red':str_red, 'sig_str_percent_blue':str_blue,
                        
                        'td_red': td_red, 'td_blue': td_blue, 'td_pct_red':td_pct_red, 'td_pct_blue':td_pct_blue,
                        'clinch_red':clinch_red, 'clinch_blue':clinch_blue, 'ground_red':ground_red, 'ground_blue':ground_blue,

                        'sub_att_red':sub_att_red, 'sub_att_blue':sub_att_blue, 'rev_red':rev_red, 'rev_blue':rev_blue,
                        'ctrl_red': ctrl_red, 'ctrl_blue': ctrl_blue,
                        
                        'head_red':head_red, 'head_blue':head_blue, 'body_red':body_red, 'body_blue':body_blue, 'leg_red':leg_red, 'leg_blue':leg_blue,
                        'distance_red':distance_red, 'distance_blue':distance_blue,
                        'total_strikes_red':strikes_red, 'total_strikes_blue':strikes_blue, 'winner':winner
                    }

                    # print(row)
                    row_df = pd.DataFrame([row])
                    global_df = pd.concat([global_df, row_df], ignore_index=True)
                    driver.back()

            if end_date:
                if event_date <= end_date:
                    return global_df  
                
        time.sleep(1)
    return global_df 




In [54]:
test_df = scrape_ufc(None, get_upcoming=False)
test_df.head()

Navigating to page 1: http://ufcstats.com/statistics/events/completed?page=1
Navigating to page 2: http://ufcstats.com/statistics/events/completed?page=2
Navigating to page 3: http://ufcstats.com/statistics/events/completed?page=3
Navigating to page 4: http://ufcstats.com/statistics/events/completed?page=4
Navigating to page 5: http://ufcstats.com/statistics/events/completed?page=5
Navigating to page 6: http://ufcstats.com/statistics/events/completed?page=6
Navigating to page 7: http://ufcstats.com/statistics/events/completed?page=7
Navigating to page 8: http://ufcstats.com/statistics/events/completed?page=8
Navigating to page 9: http://ufcstats.com/statistics/events/completed?page=9
Navigating to page 10: http://ufcstats.com/statistics/events/completed?page=10
Navigating to page 11: http://ufcstats.com/statistics/events/completed?page=11
Navigating to page 12: http://ufcstats.com/statistics/events/completed?page=12
Navigating to page 13: http://ufcstats.com/statistics/events/completed

,event_name,event_date,event_location,fight_url,red_fighter,blue_fighter,red_record,blue_record,dob_red,dob_blue,...,head_blue,body_red,body_blue,leg_red,leg_blue,distance_red,distance_blue,total_strikes_red,total_strikes_blue,winner
0,UFC Fight Night: Taira vs. Park,2025-08-02,"Las Vegas, Nevada, USA",http://ufcstats.com/fight-details/282c8c6c1ed8...,Tatsuro Taira,HyunSung Park,17-1-0,10-1-0,"Jan 27, 2000","Nov 04, 1995",...,1 of 6,0 of 0,0 of 0,1 of 1,2 of 3,4 of 7,2 of 7,13 of 20,15 of 21,Tatsuro Taira
1,UFC Fight Night: Taira vs. Park,2025-08-02,"Las Vegas, Nevada, USA",http://ufcstats.com/fight-details/426a69e34382...,Mateusz Rebecki,Chris Duncan,20-3-0,14-2-0,"Nov 03, 1992","May 10, 1993",...,67 of 181,7 of 13,20 of 35,19 of 19,5 of 6,104 of 256,85 of 214,123 of 278,95 of 225,Chris Duncan
2,UFC Fight Night: Taira vs. Park,2025-08-02,"Las Vegas, Nevada, USA",http://ufcstats.com/fight-details/878941394f9e...,Elves Brener,Esteban Ribovics,16-6-0,15-2-0,"Sep 27, 1997","Apr 27, 1996",...,86 of 233,15 of 23,23 of 29,15 of 16,13 of 19,118 of 248,114 of 268,123 of 256,123 of 284,Esteban Ribovics
3,UFC Fight Night: Taira vs. Park,2025-08-02,"Las Vegas, Nevada, USA",http://ufcstats.com/fight-details/c62ed8a6002b...,Karol Rosa,Nora Cornolle,19-7-0,9-3-0,"Dec 30, 1994","Dec 06, 1989",...,15 of 33,6 of 8,4 of 4,2 of 3,8 of 10,14 of 35,17 of 36,190 of 224,75 of 98,Karol Rosa
4,UFC Fight Night: Taira vs. Park,2025-08-02,"Las Vegas, Nevada, USA",http://ufcstats.com/fight-details/ac57250e3de8...,Neil Magny,Elizeu Zaleski dos Santos,30-14-0,25-10-1,"Aug 03, 1987","Dec 11, 1986",...,5 of 24,4 of 4,4 of 5,0 of 2,11 of 13,19 of 43,17 of 38,80 of 113,42 of 69,Neil Magny


In [56]:
test_df.to_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\all_stats_v2.csv')

In [52]:
test_df[['winner', 'performance_bonus_winner', 'red_fighter', 'blue_fighter','fight_otn_bonus', 
         'ctrl_red','sig_str_red', 'sig_str_percent_red','td_pct_red', 'clinch_red',
         'method']].tail(15)

,winner,performance_bonus_winner,red_fighter,blue_fighter,fight_otn_bonus,ctrl_red,sig_str_red,sig_str_percent_red,td_pct_red,clinch_red,method
71,Tagir Ulanbekov,0,Tagir Ulanbekov,Azat Maksum,0,3:42,68 of 118,57%,0%,11 of 13,U-DEC
72,Mohammed Usman,0,Hamdy Abdelwahab,Mohammed Usman,0,0:02,55 of 123,44%,0%,0 of 0,U-DEC
73,Kamaru Usman,0,Kamaru Usman,Joaquin Buckley,1,12:57,41 of 99,41%,30%,2 of 4,U-DEC
74,Rose Namajunas,0,Rose Namajunas,Miranda Maverick,0,6:33,25 of 86,29%,50%,1 of 1,U-DEC
75,Edmen Shahbazyan,0,Edmen Shahbazyan,Andre Petroski,0,0:15,53 of 136,38%,0%,0 of 0,U-DEC
76,Raoni Barcelos,0,Cody Garbrandt,Raoni Barcelos,0,0:46,28 of 94,29%,0%,6 of 7,U-DEC
77,DRAW,0,Mansur Abdul-Malik,Cody Brundage,0,0:07,25 of 73,34%,---,0 of 0,Overturned
78,Alonzo Menifield,0,Alonzo Menifield,Oumar Sy,0,0:00,29 of 86,33%,---,0 of 0,U-DEC
79,NC,0,Paul Craig,Rodolfo Bellato,0,0:34,17 of 27,62%,0%,2 of 3,CNC
80,Michael Chiesa,0,Michael Chiesa,Court McGee,0,1:32,72 of 160,45%,0%,20 of 28,U-DEC


TODO
calculate: sig_strike diff, 
rolling stats, for leg, distance, clinch, ground, submissions, top method of win, total rounds, fighter country 
rolling win loss record, win streaks losing streaks 
FIX REACH STATS control_pr_blue	td_pr_blue	td_defense_blue

Scraping:
fight odds: https://www.bestfightodds.com/fighters/Ian-Garry-8934
espn stats: https://www.espn.com/mma/fighter/history/_/id/4290344/austin-vanderford 

Kelly multivariate: 
https://www.cs.miami.edu/home/burt/learning/mth649.191/docs/SSRN-id2259133.pdf

Sentiment and pics: 
https://www.sherdog.com/tag/fight-odds 
https://www.tapology.com/ 